In [ ]:
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np

Retrieve Data

In [ ]:
root_dir = r'' # TODO: add results path
cnHuber = r'' # TODO: add results path
path = cnHuber
with open(os.path.join(root_dir, path,'0output.dict'), "rb") as f:
    df_results = pickle.load(f)

In [ ]:
df_results = pd.DataFrame(df_results)
df_results

End-to-End training decreases regret

In [ ]:
df_results.test_regret.plot()

Inference

In [ ]:
with open(os.path.join(root_dir, path,'pred.pkl.test0'), "rb") as f:
    df_weights = pickle.load(f)

df_weights = pd.DataFrame(df_weights)
df_weights.index.name = 'datetime'
df_weights.columns = ['instrument','score','lable','weight']
df_backtest = df_weights.reset_index().set_index(['datetime','instrument'])
df_backtest

Top k strategy for end-to-end prediction model

In [ ]:
from pprint import pprint

import qlib
import pandas as pd
from qlib.utils.time import Freq
from qlib.utils import flatten_dict
from qlib.contrib.evaluate import backtest_daily
from qlib.contrib.evaluate import risk_analysis
from qlib.contrib.strategy import TopkDropoutStrategy

# init qlib
qlib.init()

CSI300_BENCH = "SH000300"
STRATEGY_CONFIG = {
    "topk": 50,
    "n_drop": 5,
    # pred_score, pd.Series
    "signal": df_backtest.score,
}


strategy_obj = TopkDropoutStrategy(**STRATEGY_CONFIG)
report_normal, positions_normal = backtest_daily(
    start_time="2017-01-01", end_time="2020-08-01", strategy=strategy_obj
)
analysis = dict()
# default frequency will be daily (i.e. "day")
analysis["return_without_cost"] = risk_analysis(report_normal["return"])
analysis["excess_return_without_cost"] = risk_analysis(report_normal["return"] - report_normal["bench"])
analysis["excess_return_with_cost"] = risk_analysis(report_normal["return"] - report_normal["bench"] - report_normal["cost"])


analysis_df = pd.concat(analysis)  # type: pd.DataFrame
pprint(analysis_df)

Weight strategy for end-to-end prediction model

In [ ]:
from qlib.contrib.strategy import WeightStrategyBase
class Predweight(WeightStrategyBase):
    def __init__(
        self,
        **kwargs,
    ):
        super().__init__(**kwargs)

    def generate_target_weight_position(self, score, current, trade_start_time, trade_end_time):

        target_weight_position = {stock: weight for stock, weight in zip(score.index.values, score.values) if weight > 0}
        #输出是字典
        return target_weight_position

In [ ]:
from pprint import pprint
from qlib.utils.time import Freq
import pandas as pd
from qlib.backtest import backtest, executor
from qlib.contrib.evaluate import risk_analysis
from qlib.utils import flatten_dict
from qlib.contrib.strategy import TopkDropoutStrategy

STRATEGY_CONFIG = {
    "signal": df_backtest.weight,
}

EXECUTOR_CONFIG = {
    "time_per_step": "day",
    "generate_portfolio_metrics": True,
}
FREQ = "day"
backtest_config = {
    "start_time": "2017-01-01",
    "end_time": "2020-08-01",
    "account": 100000000,
    "benchmark": "SH000905",
    "exchange_kwargs": {
        "freq": FREQ,
        "limit_threshold": 0.095,
        "deal_price": "close",
        "open_cost": 0.0005,
        "close_cost": 0.0015,
        "min_cost": 5,
    },
}
strategy_obj = Predweight(**STRATEGY_CONFIG)
executor_obj = executor.SimulatorExecutor(**EXECUTOR_CONFIG)

portfolio_metric_dict, indicator_dict = backtest(executor=executor_obj,strategy=strategy_obj,**backtest_config)
analysis_freq = "{0}{1}".format(*Freq.parse(FREQ))


report_normal, positions_normal = portfolio_metric_dict.get(analysis_freq)

# analysis
analysis = dict()
analysis["return_without_cost"] = risk_analysis(
    report_normal["return"], freq=analysis_freq
)
analysis["excess_return_without_cost"] = risk_analysis(
    report_normal["return"] - report_normal["bench"], freq=analysis_freq
)
analysis["excess_return_with_cost"] = risk_analysis(
    report_normal["return"] - report_normal["bench"] - report_normal["cost"], freq=analysis_freq
)

analysis_df = pd.concat(analysis)  # type: pd.DataFrame
# log metrics
analysis_dict = flatten_dict(analysis_df["risk"].unstack().T.to_dict())
# print out results
pprint(f"The following are analysis results of benchmark return({analysis_freq}).")
pprint(risk_analysis(report_normal["bench"], freq=analysis_freq))
pprint(f"The following are analysis results of the excess return without cost({analysis_freq}).")
pprint(analysis["excess_return_without_cost"])
pprint(f"The following are analysis results of the excess return with cost({analysis_freq}).")
pprint(analysis["excess_return_with_cost"])